In [1]:
%run 1-0mmyolo-common.ipynb

/bin/bash: nvcc: command not found
gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

/bin/bash: nvidia-smi: command not found
/work/van-speech-nlp/jindaznb/j-vis
System Prefix: /work/van-speech-nlp/jindaznb/visenv
HOME: /work/van-speech-nlp/jindaznb/j-vis
2.0.0+cu117 False
3.3.0
11.7
GCC 9.3


# Write custom Config file

In [2]:
# %cd {HOME}/mmyolo
%cd {HOME}/mmdetection

/work/van-speech-nlp/jindaznb/j-vis/mmdetection


In [3]:
model_folder="sparse_rcnn"
base_model_name="sparse-rcnn_r101_fpn_300-proposals_crop-ms-480-800-3x_coco"
custom_model_name=f"custom-{base_model_name}"
CUSTOM_CONFIG_PATH = f"{HOME}/mmdetection/configs/{model_folder}/{custom_model_name}.py"
print(CUSTOM_CONFIG_PATH)


CUSTOM_CONFIG = f"""
_base_ = './{base_model_name}.py'

# ========================Frequently modified parameters======================
# -----data related-----
#model = dict(
#    roi_head=dict(
#        bbox_head=dict(num_classes={num_classes}),
#))

dataset_type = 'COCODataset'
classes =  {classes}

data_root = '{dataset_location}' # Root directory of the dataset

train_ann_file = 'train/_annotations.coco.json'  # Annotation file for training set
train_data_prefix = 'train/'  # Prefix for training data directory

val_ann_file = 'valid/_annotations.coco.json'  # Annotation file for validation set
val_data_prefix = 'valid/'  # Prefix for validation data directory

class_name = {classes} 
num_classes = {num_classes}  # Number of classes in the dataset
metainfo = dict(classes=class_name, palette=[(20, 220, 60)])  # Metadata information for visualization

train_batch_size_per_gpu = {BATCH_SIZE}  # Batch size per GPU during training
#train_num_workers = 4  # Number of worker processes for data loading during training
persistent_workers = True  # Whether to use persistent workers during training

# -----train val related-----
# base_lr = 0.004  # Base learning rate for optimization
base_lr = 0.04
max_epochs = {MAX_EPOCHS}  # Maximum training epochs
num_epochs_stage2 = 20  # Number of epochs for stage 2 training

model_test_cfg = dict(
    multi_label=True,  # Multi-label configuration for multi-class prediction
    nms_pre=30000,  # Number of boxes before NMS
    score_thr=0.001,  # Score threshold to filter out boxes
    nms=dict(type='nms', iou_threshold=0.65),  # NMS type and threshold
    max_per_img=300)  # Maximum number of detections per image


# ========================Possible modified parameters========================
default_hooks = dict(
    checkpoint=dict(
        type="CheckpointHook",
        save_best="coco/bbox_mAP_50",
        rule="greater",
        max_keep_ckpts=10,
    ),
    early_stopping=dict(
        type="EarlyStoppingHook",
        monitor="coco/bbox_mAP_50",
        patience=15,
        min_delta=0.001
    ),
)

train_cfg=dict(
    max_epochs=max_epochs
)

data = dict(
    #samples_per_gpu=1,
    #workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        img_prefix='train/',
        classes=classes,
        ann_file='train/_annotations.coco.json.json'),
    val=dict(
        type=dataset_type,
        img_prefix='valid/',
        classes=classes,
        ann_file='valid/_annotations.coco.json'),
    test=dict(
        type=dataset_type,
        img_prefix='test/',
        classes=classes,
        ann_file='test/_annotations.coco.json'))

# -----data related-----
img_scale = (1024, 1024)  # width, height
# ratio range for random resize
random_resize_ratio_range = (0.1, 2.0)
# Cached images number in mosaic
mosaic_max_cached_images = 40
# Number of cached images in mixupep
mixup_max_cached_images = 20
# Batch size of a single GPU during validation
val_batch_size_per_gpu = 8
# Worker to pre-fetch data for each single GPU during validation
val_num_workers = 3

# Config of batch shapes. Only on val.
batch_shapes_cfg = dict(
    type='BatchShapePolicy',
    batch_size=val_batch_size_per_gpu,
    img_size=img_scale[0],
    size_divisor=32,
    extra_pad_ratio=0.5)

# -----train val related-----
lr_start_factor = 1.0e-5
dsl_topk = 13  # Number of bbox selected in each level
loss_cls_weight = 1.0
loss_bbox_weight = 2.0
qfl_beta = 2.0  # beta of QualityFocalLoss
weight_decay = 0.05

# Save model checkpoint and validation intervals
save_checkpoint_intervals = 10
# validation intervals in stage 2
val_interval_stage2 = 1
# The maximum checkpoints to keep.
max_keep_ckpts = 3
# single-scale training is recommended to
# be turned on, which can speed up training.
env_cfg = dict(cudnn_benchmark=True)
"""
with open(CUSTOM_CONFIG_PATH, 'w') as file:
    file.write(CUSTOM_CONFIG)

/work/van-speech-nlp/jindaznb/j-vis/mmdetection/configs/sparse_rcnn/custom-sparse-rcnn_r101_fpn_300-proposals_crop-ms-480-800-3x_coco.py


In [ ]:
# !python tools/train.py configs/{model_folder}/{custom_model_name}.py\
#     | tee "../log/OUT_{custom_model_name}_$(date +"%Y%m%d_%H%M%S").txt"

In [5]:
CONFIDENCE_THRESHOLD = 0.5
NMS_IOU_THRESHOLD = 0.5

ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset_location}/test",
    annotations_path=f"{dataset_location}/test/_annotations.coco.json",
)

images = list(ds.images.values())
CUSTOM_WEIGHTS_PATH = glob.glob(f"{HOME}/mmdetection/work_dirs/{custom_model_name}/best_coco_bbox_mAP_50_epoch_*.pth")[0]
model = init_detector(CUSTOM_CONFIG_PATH, CUSTOM_WEIGHTS_PATH, device=DEVICE)

def callback(image: np.ndarray) -> sv.Detections:
    result = inference_detector(model, image)
    detections = sv.Detections.from_mmdetection(result)
    return detections[detections.confidence > CONFIDENCE_THRESHOLD].with_nms(threshold=NMS_IOU_THRESHOLD)


mean_average_precision = sv.MeanAveragePrecision.benchmark(
    dataset = ds,
    callback = callback
)

print('mAP:', mean_average_precision.map50_95)

Loads checkpoint by local backend from path: /work/van-speech-nlp/jindaznb/j-vis/mmdetection/work_dirs/custom-sparse-rcnn_r101_fpn_300-proposals_crop-ms-480-800-3x_coco/best_coco_bbox_mAP_50_epoch_63.pth
mAP: 0.20980822403376498


In [ ]:
ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset_location}/test",
    annotations_path=f"{dataset_location}/test/_annotations.coco.json",
)

images = list(ds.images.values())
CUSTOM_WEIGHTS_PATH = glob.glob(f"{HOME}/mmdetection/work_dirs/{custom_model_name}/best_coco_bbox_mAP_50_epoch_*.pth")[0]
model = init_detector(CUSTOM_CONFIG_PATH, CUSTOM_WEIGHTS_PATH, device=DEVICE)

best_map, best_params = optimize_params(ds, model, param_space)
print('Best mAP:', best_map)
print('Best Parameters:', best_params)

Loads checkpoint by local backend from path: /work/van-speech-nlp/jindaznb/j-vis/mmdetection/work_dirs/custom-sparse-rcnn_r101_fpn_300-proposals_crop-ms-480-800-3x_coco/best_coco_bbox_mAP_50_epoch_63.pth
|   iter    |  target   | confid... | nms_io... |
-------------------------------------------------
| 1         | 0.1995    | 0.4371    | 0.9556    |
| 2         | 0.05636   | 0.7588    | 0.6388    |
| 3         | 0.1955    | 0.2404    | 0.2404    |
| 4         | 0.1691    | 0.1523    | 0.8796    |
| 5         | 0.1976    | 0.641     | 0.7373    |
| 6         | 0.2293    | 0.7057    | 0.5777    |
| 7         | 0.07373   | 0.8456    | 0.3702    |
| 8         | 0.1826    | 0.1912    | 0.5695    |
| 9         | 0.1766    | 0.1051    | 0.5174    |
| 10        | 0.2284    | 0.7008    | 0.678     |
| 11        | 0.1955    | 0.2405    | 0.2403    |
| 12        | 0.2205    | 0.6862    | 0.709     |
| 13        | 0.2021    | 0.6585    | 0.6653    |
| 14        | 0.2151    | 0.6657    | 0.558   